In [375]:
%reset -sf

In [4]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [160]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip().split("\n")

In [161]:
# srr = """
# ...........
# .....###.#.
# .###.##..#.
# ..#.#...#..
# ....#.#....
# .##..S####.
# .##..#...#.
# .......##..
# .##.#.####.
# .##..##.##.
# ...........
# """.strip().split("\n")

In [162]:
sx, sy = 0, 0

n = len(srr)
m = len(srr[0])

arr = [[0 for _ in range(m)] for _ in range(n)]

for i,row in enumerate(srr):
    for j,cell in enumerate(row):
        if cell == "S":
            sx, sy = i, j
        if cell == "#":
            arr[i][j] = 1

In [163]:
print(n,m,sx,sy)

131 131 65 65


In [328]:
def compute(dist,threshold):

#     dist = {(sx, sy): sval}
    queue = [(v,sx,sy) for (sx,sy),v in dist.items()]
#     queue = deque(queue)

    while queue:
        _, x, y = heapq.heappop(queue)
#         x, y = queue.popleft()
        for dx, dy in d4:
            xx = x+dx
            yy = y+dy
            if -65 <= xx <= 65 and -65 <= yy <= 65:
                if arr[xx+65][yy+65] == 0:
                    if (xx, yy) not in dist:
                        dist[xx,yy] = dist[x,y] + 1
                        heapq.heappush(queue, (dist[xx,yy],xx,yy))
#                         queue.append((xx,yy))
        
    return sum(1 for (x,y),v in dist.items() if (v <= threshold and (x+y)%2 == 0))

In [329]:
compute({(0,0):0},6)

44

In [330]:
compute({(0,0):0},64)

3853

In [331]:
compute({(0,0):0},6400)

7819

In [332]:
D = 26501365

In [333]:
d2 = D // n

In [460]:
d2

202300

In [418]:
locs = (
      [(d2-x, x) for x in range(d2)]
    + [(d2-x, -x) for x in range(d2)]
    + [(-d2+x, x) for x in range(d2)]
    + [(-d2+x, -x) for x in range(d2)]
)

In [ ]:
locs

In [440]:
all_locs = []
for x,y in locs:
    for dx in [-2,-1,0,1,2]:
        for dy in [-2,-1,0,1,2]:
            all_locs.append((x+dx,y+dy))

In [396]:
all_locs = locs

In [397]:
all_locs = sorted(set(all_locs))

In [399]:
# compute(dist,D)

In [441]:
random.shuffle(all_locs)

In [401]:
# all_locs = [(d2,0), (-d2,0), (0,d2), (0,-d2)]

In [464]:
res = 0
val_set = set()

# for cx, cy in all_locs:
for cx, cy in [(d2,0), (d2,1), (d2+1,0), (d2+1,1), (d2-1,0), (d2-1,1), (d2-2,2), (d2-2,3)]:
# for cx, cy in [(d2,0), (-d2,0), (0,d2), (0,-d2)]:
# for cx, cy in [(d2+1,0), (-d2-1,0), (0,d2+1), (0,-d2-1)]:
    dist = {}
    for i in [-1,0,1]:
        for j in [-1,0,1]:
            cxx = cx * 131 + i*65
            cyy = cy * 131 + j*65
            if abs(cxx) + abs(cyy) <= D:
                dist[i*65,j*65] = abs(cxx) + abs(cyy)

    val = compute(dist,D)
    print(val)
    val_set.add(val)
    res += val
    
print(res)

5842
984
0
0
7819
6829
6829
984
29287


In [406]:
sorted(val_set)

[0, 984, 987, 990, 1005, 6814, 6829, 6832, 6833, 7819]

In [ ]:
for i in range(d2):
    cnt += 

In [422]:
locs.sort()

In [449]:
full_cnt = 0
for (a,b),(c,d) in zip(locs[::2], locs[1::2]):
    if b == d:
        continue
    cnt = d - b - 1
    assert cnt > 0
#     print(a,b,c,d,cnt)
    full_cnt += cnt

In [450]:
full_cnt

81849770802

In [445]:
# locs

In [420]:
full_cnt

-404600

In [390]:
len(locs) // 4

202300

In [411]:
res

3343153

In [462]:
(
    full_cnt * 7819
    + (202300 - 1) * sum([6814, 6829, 6832, 6833])
    + (202300) * sum([984, 987, 990, 1005])
    + 23341
)

639989684627071

In [439]:
cnt

1

In [444]:
# locs

In [287]:
d2

202300

In [261]:
compute(130,65,1,D)

7819

In [262]:
D

26501365

In [263]:
d2 * 131

26501300

In [267]:
compute(130,65,(d2-1)*131+65,D)

5960

In [241]:
len(all_locs)

6878174

In [1]:
# plt.plot([x for x,y in locs], [y for x,y in locs])

In [ ]:
# locs

In [151]:
# dist

In [78]:
26501365 / 131

202300.4961832061

In [79]:
# 26501365 / 131

In [80]:
# dist

17

In [82]:
sum(1 for (x,y),v in dist.items() if (v <= 64 and (x+y)%2 == (sx+sy)%2))

351

495

In [19]:
how many garden plots could the Elf reach in exactly 64 steps?

Object `steps` not found.


In [ ]:
how many garden plots could the Elf reach in exactly 64 steps

In [ ]:
how many garden plots could the Elf reach in exactly 64 steps

In [23]:
"""
...........
.....###.#.
.###.##.O#.
.O#O#O.O#..
O.O.#.#.O..
.##O.O####.
.##.O#O..#.
.O.O.O.##..
.##.#.####.
.##O.##.##.
...........
""".count("O")

16